In [1]:
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(r"C:\Users\gsyoume\OneDrive - Enterprise Community\EPD_WORK_ALL\2020_Quarterly_Reporting\2020\Q1\New folder (2)")

In [3]:
df = pd.read_excel('testttt.xlsx')

,ProjectID,BusinessLine,ReportingYear,ProjectName,ProjectAddress,ProjectCity,ProjectState,Unit,TDC
0,6387,Loan Fund,2013,Easton Ridge,9009 SE Causey Avenue,Happy Valley,OR,66.000000,1000.000000
1,6387,Syndication,2013,Easton Ridge,9009 SE Causey,Happy Valley,OR,66.000000,1000.000000
2,7743,Syndication,2013,Edward II Hotel,3155 Scott Street,San Francisco,CA,-122.440978,37.798991
3,6852,Loan Fund,2013,ELLF - St Bernard #2,1443 Burbank Drive,New Orleans,LA,-90.078907,30.018205
4,7464,Syndication,2013,Elmcrest Terrace,4 Elmcrest Terrace,Norwalk,CT,-73.420720,41.109039


In [6]:
df.ProjectName.value_counts()

Lake Street Studios                               3
The Greens at Liberty Road                        3
Legacy at Central Place                           2
Greens at Logan Field                             2
Greens at English Consul                          2
Easton Ridge                                      2
New Shiloh Village Apartments                     2
Kenneth Henry Court                               1
Evergreen Towers One                              1
Helen S. Brown Tower                              1
Kenyon House                                      1
Iberville Offsite Rehab 1                         1
Family Health Center of Worcester (FHCW)          1
Kings Valley Senior Apartments                    1
Harrison Ave                                      1
Habitat - St. Mark's                              1
High Place East                                   1
Gateway Science Academy South (charter school)    1
Fairmount Creamery Historic Adaptive Reuse        1
Gilliand Man

In [7]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ProjectID       43 non-null     int64  
 1   BusinessLine    43 non-null     object 
 2   ReportingYear   43 non-null     int64  
 3   ProjectName     43 non-null     object 
 4   ProjectAddress  43 non-null     object 
 5   ProjectCity     43 non-null     object 
 6   ProjectState    43 non-null     object 
 7   Unit            43 non-null     float64
 8   TDC             43 non-null     float64
dtypes: float64(2), int64(2), object(5)
memory usage: 3.1+ KB


In [8]:
df['name_address'] = df['ProjectName'] + ' ' + df['ProjectAddress']+ ' ' + df['ProjectCity'] + ' ' + df['ProjectState']
name_address = df['name_address']


In [10]:
name_address

0     Easton Ridge 9009 SE Causey Avenue Happy Valle...
1           Easton Ridge 9009 SE Causey Happy Valley OR
2     Edward II Hotel 3155 Scott Street San Francisc...
3     ELLF - St Bernard #2 1443 Burbank Drive New Or...
4        Elmcrest Terrace 4 Elmcrest Terrace Norwalk CT
5     Embassy House Apartments 2901 Dee Street Shrev...
6     Evergreen Towers One 1333 N. Cleveland Ave. Ch...
7     Fairmount Creamery Historic Adaptive Reuse 172...
8     Family Health Center of Worcester (FHCW) 26 Qu...
9     Farmington Meadows Apartments 4566 SW 160th Av...
10    Gardens at Birmingham 1460 Parkway Avenue Ewin...
11    Gateway Science Academy South (charter school)...
12    George Street Commons 13-25 East College Stree...
13    Gilliand Manor  - Green Retrofit Homes 210 Big...
14    Greens at English Consul 4120 Oak Road Lansdow...
15    Greens at English Consul 4120 Oak Road Lansdow...
16    Greens at Logan Field 3455 Dundalk Ave Dundalk MD
17    Greens at Logan Field 3455 Dundalk Ave Dun

In [9]:
vectorizer = TfidfVectorizer("char", ngram_range=(1, 4), sublinear_tf=True)
tf_idf_matrix = vectorizer.fit_transform(name_address)

C:\Users\gsyoume\Documents\Anaconda\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass input=char as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [58]:
tf_idf_matrix

<168x3236 sparse matrix of type '<class 'numpy.float64'>'
	with 6006 stored elements in Compressed Sparse Row format>

In [59]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
  
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 5)

In [60]:
matches

<168x168 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [61]:
def get_matches_df(sparse_matrix, name_vector, top=840):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

matches_df = get_matches_df(matches, name_address)

In [62]:
matches_df.to_csv('xx.csv')

In [63]:
matches_df[matches_df['similarity'] < 0.99999].sort_values(by=['similarity'], ascending=False).head(30)

,left_side,right_side,similarity
826,Pike's Place Lux Suites by Barsala 2nd Ave and...,Pike's Place Lux Suites by Barsala 2rd Ave and...,0.734016
831,Pike's Place Lux Suites by Barsala 2rd Ave and...,Pike's Place Lux Suites by Barsala 2nd Ave and...,0.734016
256,Holiday Inn Express & Suites Seattle City Cent...,Holiday Inn Express & Suites Seattle-City Cent...,0.712321
206,Holiday Inn Express & Suites Seattle-City Cent...,Holiday Inn Express & Suites Seattle City Cent...,0.712321
181,Travelodge Seattle by The Space Needle 200 6th...,Travelodge Seattle by The Space Needle 200 6th...,0.669974
211,Travelodge Seattle by The Space Needle 200 6th...,Travelodge Seattle by The Space Needle 200 6th...,0.669974
836,citizenM Seattle South Lake Union hotel 201 We...,citizenM Seattle South Lake Union hotel 201 We...,0.651961
791,citizenM Seattle South Lake Union hotel 201 We...,citizenM Seattle South Lake Union hotel 201 We...,0.651961
586,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.627400
551,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.627400


In [57]:
matches_df[matches_df['similarity'] < 0.50].right_side.nunique()

152

In [30]:
xx = matches_df.to_csv('xx.csv')

In [18]:
xx